In [1]:
import pandas as pd
from Bio import SeqIO
import glob

### metadata: genome

In [3]:
df_0 = pd.read_excel(
    "../dataset/AllSatellitesTables.xlsx", sheet_name="PICI", engine="openpyxl"
)
print(df_0.shape)
print(df_0.columns)

In [31]:
metadata = pd.read_csv("../dataset/PICI_GenomicNucInfo_060723.csv")
print(metadata.shape)
print(metadata.columns)

(1435, 6)
Index(['Unnamed: 0', 'PICI_ID', 'Host_ID', 'Host_strain', 'Host_Ref',
       'Genomic_region'],
      dtype='object')


In [72]:
metadata["PICI_ID_allsets"] = metadata["PICI_ID"].str.split(".Set", n=1).str[0]
cols = ["PICI_ID_allsets"] + [
    col for col in metadata.columns if col != "PICI_ID_allsets"
]
metadata = metadata[cols]
metadata = metadata.drop(columns=["Unnamed: 0"])

In [73]:
metadata

,PICI_ID_allsets,PICI_ID,Host_ID,Host_strain,Host_Ref,Genomic_region
0,STAU002.0321.00050.C001.PICI.TypeB.variant0002,STAU002.0321.00050.C001.PICI.TypeB.variant0002...,STAU002.0321.00050.C001,Staphylococcus aureus strain HOU1444-VR chromo...,GCF_001278745.1,"(401962, 412834)"
1,POSP014.0321.00001.C001.PICI.TypeB.variant0001,POSP014.0321.00001.C001.PICI.TypeB.variant0001...,POSP014.0321.00001.C001,"Polaromonas sp. JS666, complete sequence.",GCF_000013865.1,"(3126147, 3132812)"
2,STAU002.0321.00250.C001.PICI.TypeB.variant0002,STAU002.0321.00250.C001.PICI.TypeB.variant0002...,STAU002.0321.00250.C001,Staphylococcus aureus subsp. aureus strain 80w...,GCF_003944865.1,"(814785, 826925)"
3,STAU002.0321.00374.C001.PICI.TypeB.variant0002,STAU002.0321.00374.C001.PICI.TypeB.variant0002...,STAU002.0321.00374.C001,Staphylococcus aureus strain UP_1150 chromosom...,GCF_009912535.1,"(2745923, 2756343)"
4,MYTU002.0321.00238.C001.PICI.TypeB.variant0004,MYTU002.0321.00238.C001.PICI.TypeB.variant0004...,MYTU002.0321.00238.C001,Mycobacterium tuberculosis strain DKC2 isolate...,GCF_900520315.1,"(105183, 113045)"
...,...,...,...,...,...,...
1430,STSA003.0321.00008.C001.PICI.TypeB.variant0002,STSA003.0321.00008.C001.PICI.TypeB.variant0002...,STSA003.0321.00008.C001,Staphylococcus saprophyticus strain UTI-056 ch...,GCF_013341435.1,"(2129, 11546)"
1431,STSI001.0321.00003.C001.PICI.TypeB.variant0001,STSI001.0321.00003.C001.PICI.TypeB.variant0001...,STSI001.0321.00003.C001,"Staphylococcus simulans strain MR3, complete s...",GCF_003006055.1,"(1027781, 1037385)"
1432,STSI001.0321.00004.C001.PICI.TypeB.variant0001,STSI001.0321.00004.C001.PICI.TypeB.variant0001...,STSI001.0321.00004.C001,"Staphylococcus simulans strain MR4, complete s...",GCF_003006075.1,"(1027781, 1037385)"
1433,STSI001.0321.00005.C001.PICI.TypeB.variant0001,STSI001.0321.00005.C001.PICI.TypeB.variant0001...,STSI001.0321.00005.C001,"Staphylococcus simulans strain MR2, complete s...",GCF_003076375.1,"(1027882, 1037486)"


In [74]:
duplicated_sets = metadata.groupby("PICI_ID_allsets")["PICI_ID"].nunique()
duplicated_sets = duplicated_sets[duplicated_sets > 1].index
duplicate_rows = metadata[metadata["PICI_ID_allsets"].isin(duplicated_sets)]
duplicate_rows = duplicate_rows.sort_values(["PICI_ID_allsets", "PICI_ID"])

In [79]:
duplicate_rows.to_csv("../dataset/duplicate_picis.csv", index=False)

In [77]:
print(len(metadata["PICI_ID_allsets"].unique()))
print(len(metadata["PICI_ID"].unique()))
print(len(duplicate_rows["PICI_ID_allsets"].unique()))
print(len(duplicate_rows["PICI_ID"].unique()))
# 1435 - 1286 = 287 - 138 = 149

1286
1435
138
287


### metadata: protein

In [22]:
seqs = list(SeqIO.parse("../dataset/PICI_NucSequences_060723.fst", "fasta"))
print(len(seqs))
print("first sample:")
print(seqs[0])

1435
first sample:
ID: STAU002.0321.00050.C001.PICI.TypeB.variant0002.Set1
Name: STAU002.0321.00050.C001.PICI.TypeB.variant0002.Set1
Description: STAU002.0321.00050.C001.PICI.TypeB.variant0002.Set1
Number of features: 0
Seq('TAACTCCCTATCCCCTCTAATTTATTCATCATATCTTGTGCCATTTTATCCGTA...TGA')


In [29]:
seqs_prt_1 = list(
    SeqIO.parse(
        "../dataset/SatelliteProteomes/ACER001.0321.00001.C001.PICI.TypeB.variant0001.Set1.prt",
        "fasta",
    )
)
print(len(seqs_prt_1))
print(seqs_prt_1[0])

8
ID: ACER001.0321.00001.C001.PICI.TypeB.variant0001.Set1_00
Name: ACER001.0321.00001.C001.PICI.TypeB.variant0001.Set1_00
Description: ACER001.0321.00001.C001.PICI.TypeB.variant0001.Set1_00 ACER001.0321.00001.C001_00176 C ATG TGA 197540 198730 Valid CDG81_RS00890 1191 @WP_084133721.1@ CDG81_RS00890 1 197540 198730 | site-specific integrase  (translation)
Number of features: 0
Seq('MARVWIEDRSSHSGYKQALAQAKTSRKAAPGRWRVRWYDQEGKPKSQTFSRKPD...DQM')


In [2]:
data_prt = []

prt_files = glob.glob("../dataset/SatelliteProteomes/*.prt")

for prt_file in prt_files:
    sequences = list(SeqIO.parse(prt_file, "fasta"))

    for seq in sequences:
        # Extract function (part after the last '|')
        function = (
            seq.description.split("|")[-1].strip() if "|" in seq.description else ""
        )

        # Extract PICI_ID (everything before the last underscore in Protein_ID)
        pici_id = "_".join(seq.id.split("_")[:-1])

        # Create dictionary for this entry
        entry = {
            "PICI_ID": pici_id,
            "Protein_ID": seq.id,
            "Function": function,
            "Description": seq.description,
        }

        data_prt.append(entry)

df_prt = pd.DataFrame(data_prt)

In [38]:
df_prt["PICI_ID_allsets"] = df_prt["PICI_ID"].str.split(".Set", n=1).str[0]
cols = ["PICI_ID_allsets"] + [col for col in df_prt.columns if col != "PICI_ID_allsets"]
df_prt = df_prt[cols]

In [40]:
df_prt.to_csv("../dataset/PICI_proteins.csv", index=False)

In [3]:
df_prt = pd.read_csv("../dataset/PICI_proteins.csv")

In [4]:
df_prt.head()

,PICI_ID_allsets,PICI_ID,Protein_ID,Function,Description
0,MYTU002.0321.00124.C001.PICI.TypeB.variant0003,MYTU002.0321.00124.C001.PICI.TypeB.variant0003...,MYTU002.0321.00124.C001.PICI.TypeB.variant0003...,site-specific integrase (translation),MYTU002.0321.00124.C001.PICI.TypeB.variant0003...
1,MYTU002.0321.00124.C001.PICI.TypeB.variant0003,MYTU002.0321.00124.C001.PICI.TypeB.variant0003...,MYTU002.0321.00124.C001.PICI.TypeB.variant0003...,hypothetical protein (translation),MYTU002.0321.00124.C001.PICI.TypeB.variant0003...
2,MYTU002.0321.00124.C001.PICI.TypeB.variant0003,MYTU002.0321.00124.C001.PICI.TypeB.variant0003...,MYTU002.0321.00124.C001.PICI.TypeB.variant0003...,helix-turn-helix domain-containing protein (t...,MYTU002.0321.00124.C001.PICI.TypeB.variant0003...
3,MYTU002.0321.00124.C001.PICI.TypeB.variant0003,MYTU002.0321.00124.C001.PICI.TypeB.variant0003...,MYTU002.0321.00124.C001.PICI.TypeB.variant0003...,DUF2742 domain-containing protein (translation),MYTU002.0321.00124.C001.PICI.TypeB.variant0003...
4,MYTU002.0321.00124.C001.PICI.TypeB.variant0003,MYTU002.0321.00124.C001.PICI.TypeB.variant0003...,MYTU002.0321.00124.C001.PICI.TypeB.variant0003...,DUF3631 domain-containing protein (translation),MYTU002.0321.00124.C001.PICI.TypeB.variant0003...


In [43]:
print(len(df_prt["PICI_ID_allsets"].unique()))
print(len(df_prt["PICI_ID"].unique()))
print(len(df_prt["Protein_ID"].unique()))
print(len(df_prt["Description"].unique()))
print(len(df_prt["Function"].unique()))

1287
1435
20594
20594
287


### protein function

In [26]:
function_counts = df_prt["Function"].value_counts().reset_index(name="Count")

In [27]:
function_counts

,Function,Count
0,hypothetical protein (translation),8173
1,site-specific integrase (translation),984
2,terminase small subunit (translation),797
3,helix-turn-helix transcriptional regulator (t...,764
4,helix-turn-helix domain-containing protein (t...,746
...,...,...
282,DUF2004 domain-containing protein (translation),1
283,TrmB family transcriptional regulator (transl...,1
284,antitermination protein (translation),1
285,ORF6N domain-containing protein (translation),1


In [28]:
integrase_count = function_counts[function_counts["Function"].str.contains("integrase")]
integrase_count.to_csv("../dataset/proteins_integrases.csv", index=False)
alpa_count = function_counts[function_counts["Function"].str.contains("AlpA")]
alpa_count.to_csv("../dataset/proteins_AlpA.csv", index=False)
capsid_count = function_counts[function_counts["Function"].str.contains("capsid")]
capsid_count.to_csv("../dataset/proteins_capsid.csv", index=False)

In [5]:
# group by PICI_IDs and aggregate all functions for each PICI
pici_functions = df_prt.groupby("PICI_ID")["Function"].agg(list)


# Create a function to check our conditions
def check_functions(func_list):
    func_str = " ".join(func_list).lower()
    return ("integrase" in func_str) and ("alpa" in func_str)


# and ('capsid' not in func_str)

# Filter PICIs that meet our criteria
filtered_picis = pici_functions[pici_functions.apply(check_functions)]

# Create a new DataFrame with PICI_IDs and their functions
result_df = pd.DataFrame(
    {"PICI_ID": filtered_picis.index, "All_Functions": filtered_picis.values}
)

In [54]:
result_df.to_csv("../results/picis_integrase_alpa.csv", index=False)

In [6]:
function_matrix = pd.pivot_table(
    df_prt,
    values="Protein_ID",  # use Protein_ID just for counting
    index="PICI_ID",
    columns="Function",
    aggfunc="count",
    fill_value=0,
)

In [7]:
function_matrix.to_csv("../results/protein_function_matrix_old.csv", index=True)

In [8]:
function_matrix.sum().sum()

np.int64(20594)

In [ ]:
col_mask = function_matrix.columns.str.contains("integrase|alpa|capsid", case=False)
function_matrix_integrase_alpa_capsid = function_matrix.loc[:, col_mask]

In [ ]:
function_matrix_integrase_alpa_capsid.to_csv(
    "../results/protein_function_matrix_integrase_alpa_capsid.csv", index=True
)

In [129]:
function_matrix_integrase_alpa_capsid.columns

Index(['4.4 phage-related function phage major capsid protein  (translation)',
       '4.4 phage-related function site-specific integrase  (translation)',
       'AlpA family phage regulatory protein  (translation)',
       'AlpA family transcriptional regulator  (translation)',
       'DDE-type integrase/transposase/recombinase  (translation)',
       'Integrase site-specific integrase  (translation)',
       'Integrase tyrosine-type recombinase/integrase  (translation)',
       'capsid morphogenesis B protein  (translation)',
       'integrase  (translation)',
       'integrase arm-type DNA-binding domain-containing protein  (translation)',
       'integrase family protein  (translation)',
       'major capsid protein  (translation)',
       'minor capsid protein  (translation)',
       'phage capsid protein  (translation)', 'phage integrase  (translation)',
       'phage integrase Arm DNA-binding domain-containing protein  (translation)',
       'phage major capsid protein  (transla